In [1]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
pn.extension('tabulator')

import hvplot.pandas

In [2]:
sample = pd.read_csv("C:\\Users\Lloyd\Documents\GitHub\DataAnalyticsFirst\GlobalWeatherRepository.csv")
sample['last_updated'] = pd.to_datetime(sample['last_updated'])
sample

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1693301400,2023-08-29 14:00:00,28.8,83.8,Sunny,...,7.9,11.1,1,1,05:24 AM,06:24 PM,05:39 PM,02:48 AM,Waxing Gibbous,93
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1693301400,2023-08-29 11:30:00,27.0,80.6,Partly cloudy,...,28.2,29.6,2,3,06:04 AM,07:19 PM,06:50 PM,03:25 AM,Waxing Gibbous,93
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1693301400,2023-08-29 10:30:00,28.0,82.4,Partly cloudy,...,6.4,7.9,1,1,06:16 AM,07:21 PM,06:46 PM,03:50 AM,Waxing Gibbous,93
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1693301400,2023-08-29 11:30:00,10.2,50.4,Sunny,...,0.5,0.8,1,1,07:16 AM,08:34 PM,08:08 PM,04:38 AM,Waxing Gibbous,93
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1693301400,2023-08-29 10:30:00,25.0,77.0,Partly cloudy,...,139.6,203.3,4,10,06:11 AM,06:06 PM,04:43 PM,04:41 AM,Waxing Gibbous,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6226,Venezuela,Caracas,10.50,-66.92,America/Caracas,1695941100,2023-09-28 18:45:00,30.0,86.0,Partly cloudy,...,26.2,27.8,2,3,06:16 AM,06:20 PM,05:59 PM,05:25 AM,Waxing Gibbous,97
6227,Vietnam,Hanoi,21.03,105.85,Asia/Bangkok,1695940200,2023-09-29 05:30:00,24.0,75.2,Partly cloudy,...,67.4,74.2,4,9,05:47 AM,05:48 PM,05:11 PM,04:18 AM,Waxing Gibbous,97
6228,Yemen,Sanaa,15.35,44.21,Asia/Aden,1695941100,2023-09-29 01:45:00,16.1,61.0,Clear,...,33.4,142.8,2,3,05:53 AM,05:55 PM,05:22 PM,04:40 AM,Waxing Gibbous,97
6229,Zambia,Lusaka,-15.42,28.28,Africa/Lusaka,1695941100,2023-09-29 00:45:00,25.0,77.0,Clear,...,12.2,17.5,1,2,05:52 AM,06:03 PM,05:17 PM,05:05 AM,Waxing Gibbous,97


In [3]:
isample = sample.interactive()

# Line plot for temperature and time (JL)

In [4]:
# Slider widget
slider = pn.widgets.IntSlider(name='Data slider', start=1, end=100, step=1, value=1)
slider

IntSlider(end=100, name='Data slider', start=1, value=1)

In [5]:
datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    name='Datetime Range Slider',
    start=dt.datetime(2023, 9, 5,14,0,0), end=dt.datetime(2023, 9, 30,0,0),
    value=(dt.datetime(2023, 9, 8,0,0), dt.datetime(2023, 9, 28,0,0)),
    step=10000
)

datetime_range_slider

DatetimeRangeSlider(end=datetime.datetime(2023, ..., name='Datetime Range Slider', start=datetime.datetime(2023, ..., step=10000, value=(datetime.datetime(2023, ..., value_end=datetime.datetime(2023, ..., value_start=datetime.datetime(2023, ...)

In [6]:
# Radio buttons for CO2 measures
radio = pn.widgets.RadioButtonGroup(
    name='Temperature', 
    options=['temperature_celsius', 'temperature_fahrenheit'],
    button_type='success'
)
countries = ['Philippines','United States of America','Canada','Germany','China','Japan','United Kingdom','Australia','India','Russia']

In [7]:
temperature_time_pipeline = (isample[(isample.moon_illumination <= slider) & (isample.country.isin(countries))]
    .groupby(['country','location_name','last_updated'])[radio].max()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

temperature_time_pipeline

In [8]:
temperature_time_lineplot = temperature_time_pipeline.hvplot(x='last_updated',by='country',y=radio,line_width=2,title="Temperature by Country")
temperature_time_lineplot

# Scatterplot and table for air quality and location (JL)

In [9]:
airquality_location_pipeline = (isample[(isample.moon_illumination <= slider) & (isample.country.isin(countries))]
    .groupby(['country','location_name','air_quality_PM10'])['air_quality_PM2.5'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

In [10]:
airquality_location_table = airquality_location_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 15, sizing_mode='stretch_width') 
airquality_location_table

In [11]:
airquality_location_scatterplot = airquality_location_pipeline.hvplot(x='country',y='air_quality_PM2.5',by='location_name',
                                                                    size=80,kind='scatter',
                                                                    alpha=0.5,
                                                                    legend=True,
                                                                    height=500,
                                                                    width=900,
                                                                    )

airquality_location_scatterplot

# Bar chart

In [12]:
air_index_radio = pn.widgets.RadioButtonGroup(
    name='Air quality index', 
    options=['air_quality_us-epa-index','air_quality_gb-defra-index'], 
    button_type='success'
)

In [13]:
air_index_pipeline = (isample[(isample.moon_illumination <= slider) & (isample.country.isin(countries))]
    .groupby(['country'])[air_index_radio].max()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

air_index_pipeline

In [14]:
air_index_barplot = air_index_pipeline.hvplot(kind='bar',x='country',y=air_index_radio,title='Air Quality Index')
air_index_barplot

# Pie Chart About Weather Conditions (Gab)

In [15]:
# Assuming 'sample' is your original DataFrame
weather_column = ['country', 'condition_text']
weather = sample[weather_column]

# Define the countries to show
countries_to_show = ['Philippines', 'United States of America', 'Canada', 'Germany', 'China', 'Japan', 
                     'United Kingdom', 'Australia', 'Switzerland', 'India', 'Russia']

# Create a dropdown widget
country_dropdown = widgets.Dropdown(options=countries_to_show, description='Select a country:')
output_plot = widgets.Output()

In [16]:
# Define a function to generate or update the pie chart based on the selected country
def generate_pie_chart(selected_country):
    with output_plot:
        # Clear previous output
        clear_output(wait=True)
        
        # Filter the DataFrame for the selected country
        filtered_data = weather[weather['country'] == selected_country]
        condition_counts = filtered_data['condition_text'].value_counts()

        # Plot a pie chart with a fixed size
        plt.figure(figsize=(8, 6))  # Adjust the size as needed
        plt.pie(condition_counts, labels=condition_counts.index, autopct='%1.1f%%', startangle=140)
        plt.title(f'Weather Condition Distribution in {selected_country}\n\n')
        plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        plt.show()

In [17]:
# Define a function to update the pie chart when the dropdown value changes
def on_country_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        generate_pie_chart(change['new'])

# Call the function to generate the initial pie chart
generate_pie_chart(countries_to_show[0])

# Call the function when the dropdown value changes
country_dropdown.observe(on_country_dropdown_change)

# Display the dropdown and the pie chart below it
display(widgets.VBox([country_dropdown, output_plot]))

# Create dashboard

In [22]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Global Weather Report Update', 
    sidebar=[pn.pane.Markdown("# World Weather Repository ( Daily Updating )"), 
             pn.pane.Markdown("This dataset provides daily weather information for capital cities around the world. Unlike forecast data, this dataset offers a comprehensive set of features that reflect the current weather conditions around the world."), 
             pn.pane.PNG('https://storage.googleapis.com/kaggle-datasets-images/3678699/6383099/61a1c18c08506e236bf0cd0394c09110/dataset-cover.jpg?t=2023-08-29-10-36-14', sizing_mode='scale_both'),],
    main=[pn.Row(pn.Column(airquality_location_scatterplot.panel(width=600), margin=(0,25)),
                airquality_location_table.panel(width=600)),
         pn.Row(pn.Column(temperature_time_lineplot.panel(height=500),margin=(0,25)),
               air_index_barplot.panel(height=500))],
    accent_base_color="#88d8b0",
    header_background="#444444",
)
template.show()
template.servable();

Launching server at http://localhost:62296
